### Necessary Packages

In [13]:
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import sys
import os
import warnings
sys.path.append('../..')
from itertools import product
from tqdm.auto import tqdm
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,TargetEncoder,FunctionTransformer,StandardScaler
from sklearn.model_selection import GroupKFold,StratifiedGroupKFold
from sklearn.impute import SimpleImputer,KNNImputer,MissingIndicator
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA,TruncatedSVD
from src.utils import evaluate,seed_everything,plot_confusion_matrix,score
from definitions import *
from sklearn.ensemble import VotingClassifier
warnings.filterwarnings('ignore',category=UserWarning)

### Constants

In [2]:
SEED = 42

In [3]:
seed_everything(SEED)

### Utils

In [4]:
df = pd.read_csv(os.path.join(ISIS_2024_DIR,'metadata.csv'))

/tmp/ipykernel_91276/487156364.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(ISIS_2024_DIR,'metadata.csv'))


- Feature engeering

In [5]:
def feature_engineering(df : pd.DataFrame) -> pd.DataFrame:

    eps = 1e-6

    new_features = pd.DataFrame()
    
    df["color_variance_ratio"]           = df["tbp_lv_color_std_mean"] / (df["tbp_lv_stdLExt"] + eps)
    df["border_color_interaction"]       = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    df["size_color_contrast_ratio"]      = df["clin_size_long_diam_mm"] / (df["tbp_lv_deltaLBnorm"] + eps)
    df["age_normalized_nevi_confidence"] = df["tbp_lv_nevi_confidence"] / (df["age_approx"] + eps)
    df["color_asymmetry_index"]          = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    df["3d_volume_approximation"]        = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    df["color_range"]                    = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs() + (df["tbp_lv_A"] - df["tbp_lv_Aext"]).abs() + (df["tbp_lv_B"] - df["tbp_lv_Bext"]).abs()
    df["shape_color_consistency"]        = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    df["border_length_ratio"]            = df["tbp_lv_perimeterMM"] / (2 * np.pi * np.sqrt(df["tbp_lv_areaMM2"] / np.pi))
    df["age_size_symmetry_index"]        = df["age_approx"] * df["clin_size_long_diam_mm"] * df["tbp_lv_symm_2axis"]
    df["age_size_symmetry_index2"]       = df["age_approx"] * df["tbp_lv_areaMM2"] * df["tbp_lv_symm_2axis"]

    df["lesion_size_ratio"]              = df["tbp_lv_minorAxisMM"] / (df["clin_size_long_diam_mm"] + eps)
    df["lesion_shape_index"]             = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2 + eps)
    df["hue_contrast"]                   = (df["tbp_lv_H"] - df["tbp_lv_Hext"]).abs()
    df["luminance_contrast"]             = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs()
    df["lesion_color_difference"]        = np.sqrt(df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2)
    df["border_complexity"]              = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    df["color_uniformity"]               = df["tbp_lv_color_std_mean"] / (df["tbp_lv_radial_color_std_max"] + eps)
    df["3d_position_distance"]           = np.sqrt(df["tbp_lv_x"] ** 2 + df["tbp_lv_y"] ** 2 + df["tbp_lv_z"] ** 2) 
    df["perimeter_to_area_ratio"]        = df["tbp_lv_perimeterMM"] / (df["tbp_lv_areaMM2"] + eps)
    df["lesion_visibility_score"]        = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"]
    df["combined_anatomical_site"]       = df["anatom_site_general"] + "_" + df["tbp_lv_location"]
    df["symmetry_border_consistency"]    = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    df["color_consistency"]              = df["tbp_lv_stdL"] / (df["tbp_lv_Lext"] + eps)
    
    df["size_age_interaction"]           = df["clin_size_long_diam_mm"] * df["age_approx"]
    df["hue_color_std_interaction"]      = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    df["lesion_severity_index"]          = (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"] + df["tbp_lv_eccentricity"]) / 3
    df["shape_complexity_index"]         = df["border_complexity"] + df["lesion_shape_index"]
    df["color_contrast_index"]           = df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"] + df["tbp_lv_deltaLBnorm"]
    df["log_lesion_area"]                = np.log(df["tbp_lv_areaMM2"] + 1)
    df["normalized_lesion_size"]         = df["clin_size_long_diam_mm"] / df["age_approx"]
    df["mean_hue_difference"]            = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    df["std_dev_contrast"]               = np.sqrt((df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2) / 3)
    df["color_shape_composite_index"]    = (df["tbp_lv_color_std_mean"] + df["tbp_lv_area_perim_ratio"] + df["tbp_lv_symm_2axis"]) / 3
    df["3d_lesion_orientation"]          = np.arctan2(df["tbp_lv_y"], df["tbp_lv_x"])
    df["overall_color_difference"]       = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"]) / 3
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    df["comprehensive_lesion_index"]     = (df["tbp_lv_area_perim_ratio"] + df["tbp_lv_eccentricity"] + df["tbp_lv_norm_color"] + df["tbp_lv_symm_2axis"]) / 4

    return df

In [6]:
class Cat2CatInteraction(BaseEstimator, TransformerMixin):

    def __init__(self,cat_cols : np.ndarray) -> None:
        self.cat_cols = cat_cols

    def fit(self, X, y=None) -> 'Cat2CatInteraction':

        print('Fitting Cat2CatInteraction...')

        cat_cols = self.cat_cols

        transformer_list = []

        for col in cat_cols:

            others = cat_cols[cat_cols != col]
            target_type = 'binary' if np.unique(X[:,col]).shape[0] == 2 else 'multiclass'

            transformer = TargetEncoder(target_type=target_type)
            
            transformer.fit(X[:,others], X[:,col])

            transformer_list.append((f'{col}2col', transformer))

        self.transformer_list = transformer_list

        return self

    def transform(self, X) -> np.ndarray:

        features = []

        for col,(_, transformer) in zip(self.cat_cols,self.transformer_list):

            other_cols = self.cat_cols[self.cat_cols != col]

            features.append(transformer.transform(X[:,other_cols]))

        return np.column_stack(features)

In [7]:
class Cat2NumInteraction(BaseEstimator, TransformerMixin):

    def __init__(self,cat_cols : np.ndarray,num_cols : np.ndarray) -> None:
        self.cat_cols = cat_cols
        self.num_cols = num_cols

    def fit(self, X, y=None) -> 'Cat2NumInteraction':

        print('Fitting Cat2NumInteraction...')

        cat_cols = self.cat_cols
        num_cols = self.num_cols

        transformer_list = []
            
        for num_col in num_cols:
            
            transformer = TargetEncoder(target_type='continuous')
        
            transformer.fit(X[:,cat_cols], X[:,num_col])
        
            transformer_list.append((f'cat_cols2{num_col}', transformer))
    
        self.transformer_list = transformer_list

        return self

    def transform(self, X) -> np.ndarray:
        
        features = []
        
        for _, transformer in self.transformer_list:
            features.append(transformer.transform(X[:,self.cat_cols]))
        
        return np.column_stack(features)

- Data preparation

In [8]:
isic_cols = ['age_approx','sex','anatom_site_general','clin_size_long_diam_mm','tbp_lv_A']
isic_cols += ['tbp_lv_Aext','tbp_lv_B','tbp_lv_Bext','tbp_lv_C','tbp_lv_Cext','tbp_lv_H','tbp_lv_Hext','tbp_lv_L']
isic_cols += ['tbp_lv_Lext','tbp_lv_areaMM2','tbp_lv_area_perim_ratio','tbp_lv_color_std_mean','tbp_lv_deltaA']
isic_cols += ['tbp_lv_deltaB','tbp_lv_deltaL','tbp_lv_deltaLB','tbp_lv_deltaLBnorm','tbp_lv_eccentricity','tbp_lv_location']
isic_cols += ['tbp_lv_minorAxisMM','tbp_lv_nevi_confidence','tbp_lv_norm_border','tbp_lv_norm_color']
isic_cols += ['tbp_lv_perimeterMM','tbp_lv_radial_color_std_max','tbp_lv_stdL','tbp_lv_stdLExt','tbp_lv_symm_2axis']
isic_cols += ['tbp_lv_symm_2axis_angle','tbp_lv_x','tbp_lv_y','tbp_lv_z','tbp_lv_location_simple']
isic_cols += ['target','patient_id']

In [17]:
def prepare_data():

    ### Data loading
    print('Loading data...')
    df = pd.read_csv(os.path.join(ISIS_2024_DIR,'metadata.csv'))
    features_df = pd.read_csv(os.path.join(EXPIREMENTS_DIR,'vit_resnet','features.csv'))

    ### Pick the columns we want to use
    df = df[isic_cols + ['isic_id']]

    ### Merge the features
    gfk = StratifiedGroupKFold(n_splits=5)

    """features = []

    for fold,(train_idx,val_idx) in enumerate(splits):
        cols = features_df.columns[features_df.columns.str.contains(f'fold_{fold}_')].to_list() + ['isic_id']
        f = features_df.iloc[val_idx][cols].copy()
        f.columns = f.columns.str.replace(f'fold_{fold}_','')
        features.append(f)

    features = pd.concat(features)
    df = df.merge(features,on='isic_id',how='left')"""

    ### Feature engineering
    print('Feature engineering...')
    df = feature_engineering(df)
    
    ### Preprocessing
    print('Preprocessing...')
    df = df.drop(columns=['isic_id'])

    X = df.drop(columns=['target','patient_id'])
    y = df['target']

    splits = list(gfk.split(np.arange(len(df)),y=y,groups=df['patient_id']))

    X_num = X.select_dtypes(np.number)
    X_cat = X.select_dtypes(object)
    X = pd.concat([X_num,X_cat],axis=1)

    print(X.shape)

    cat_indices = np.argwhere(X.columns.isin(X.select_dtypes(object).columns)).flatten()
    num_indices = np.argwhere(X.columns.isin(X.select_dtypes(np.number).columns)).flatten()

    print(len(cat_indices),len(num_indices))

    preprocessor = Pipeline([
        ('preprocessing',ColumnTransformer([
            ('numerical_processing',SimpleImputer(strategy='mean',add_indicator=True),X.select_dtypes(np.number).columns),
            ('categorical_processing',OrdinalEncoder(handle_unknown='use_encoded_value',encoded_missing_value=-1,unknown_value=-2),X.select_dtypes(object).columns)
        ],remainder='passthrough'))
    ])

    preprocessor = preprocessor.fit(X,y)
    X = preprocessor.transform(X)

    ### Data splitting
    print('Splitting data...')
    
    data = []

    for fold,(train_idx,val_idx) in enumerate(splits):

        fold_feature = features_df[f'fold_{fold}_rotate_0'].values
        new_X = np.column_stack([X,fold_feature])

        X_train = new_X[train_idx,:]
        y_train = y[train_idx]

        X_val = new_X[val_idx,:]
        y_val = y[val_idx]

        data.append({
            'train' :  xgb.DMatrix(X_train,label=y_train),# (X_train,y_train,lesion_id_train)
            'val' : xgb.DMatrix(X_val,label=y_val), # (X_val,y_val,lesion_id_val)
        })

    return data,preprocessor,features_df

In [18]:
data,preprocessor,features = prepare_data()

Loading data...


/tmp/ipykernel_91276/3356300212.py:5: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(ISIS_2024_DIR,'metadata.csv'))


Feature engineering...
Preprocessing...
(401059, 76)
5 71
Splitting data...


In [20]:
for fold in data:
    print(fold['train'].get_label().sum())

316.0
310.0
315.0
315.0
316.0


In [11]:
data[0]['train'].num_col()

83

- Training

In [12]:
class XGBEnsembler:

    def __init__(self, params : list[dict]) -> None:
        
        self.params = params
        self.models = None


    def fit(self,data : xgb.DMatrix) -> None:

        self.models = []
        
        for param in self.params:
            model = xgb.train(param,data)
            self.models.append(model)

    def predict(self,data : xgb.DMatrix) -> np.ndarray:
        
        preds = np.zeros((data.num_row(),len(self.models)))

        for i,model in enumerate(self.models):
            preds[:,i] = model.predict(data)

        preds = np.mean(preds,axis=1)

        return preds

In [13]:
top_k_xgb_models = {}
k = 5

In [14]:
def objective(trial : optuna.Trial):

    params = {
        "seed" : SEED,
        "device" : "gpu",
        "verbosity" : 0,
    }

    params["n_estimators"] = trial.suggest_int("n_estimators", 100, 5000)

    params["objective"] = "binary:logistic"

    params["eval_metric"] = "logloss"

    params["eta"] = trial.suggest_float("eta", 0.01, 0.3)

    params["max_depth"] = trial.suggest_int("max_depth", 3, 128)

    params["subsample"] = trial.suggest_float("subsample", 0.3, 1.0)

    params["colsample_bytree"] = trial.suggest_float("colsample_bytree", 0.3, 1.0)

    params["scale_pos_weight"] = trial.suggest_float("scale_pos_weight", 1.0, 1200.0)

    params["min_child_weight"] = trial.suggest_int("min_child_weight", 1, 500)

    params["gamma"] = trial.suggest_float("gamma", 0.0, 10.0)
    params["lambda"] = trial.suggest_float("lambda", 0.0, 10.0)
    
    scores = []
    models = []

    for fold in data:

        dtrain = fold['train']
        dval = fold['val']
        model = xgb.train(params,dtrain)
        y_hat = model.predict(dval)
        
        models.append(model)
        scores.append(score(dval.get_label(), y_hat))

    mean = np.mean(scores)
    

    if len(top_k_xgb_models) < k:
        top_k_xgb_models[mean] = (params,models)
    else:
        min_score = min(top_k_xgb_models.keys())
        if mean > min_score:
            del top_k_xgb_models[min_score]
            top_k_xgb_models[mean] = (params,models)

    return mean

In [15]:
"""def objective(trial: optuna.Trial) -> float:

    params = {
        "boosting": "gbdt",
        "objective": "binary",
        "num_leaves": trial.suggest_int("num_leaves", 2, 32),
        "max_depth": trial.suggest_int("max_depth", 2, 32),
        "num_iterations": trial.suggest_int("n_estimators", 100, 1500),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 500),
        "subsample": trial.suggest_float("subsample", 0.4, 1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        "random_state" : SEED,
        "verbosity" : -1,
        "device" : "gpu",
        "class_weight" : {
            0 : 1,
            1 : trial.suggest_float("class_weight", 1, 1200)
        },
    }

    scores = []

    for fold in data:

        X_train,y_train,l_train = fold['train']
        X_test,y_test,l_test = fold['val']

        model = lgb.LGBMClassifier(**params)
        model = model.fit(X_train,y_train)

        # model.fit(X_train, y_train)

        y_hat = model.predict_proba(X_test)[:,1]

        pauc = score(y_test, y_hat)

        scores.append(pauc)

    mean = np.mean(scores)

    if len(top_k_xgb_models) < k:
        top_k_xgb_models[mean] = params
    else:
        min_score = min(top_k_xgb_models.keys())
        if mean > min_score:
            del top_k_xgb_models[min_score]
            top_k_xgb_models[mean] = params

    return mean"""

'def objective(trial: optuna.Trial) -> float:\n\n    params = {\n        "boosting": "gbdt",\n        "objective": "binary",\n        "num_leaves": trial.suggest_int("num_leaves", 2, 32),\n        "max_depth": trial.suggest_int("max_depth", 2, 32),\n        "num_iterations": trial.suggest_int("n_estimators", 100, 1500),\n        "min_child_samples": trial.suggest_int("min_child_samples", 5, 500),\n        "subsample": trial.suggest_float("subsample", 0.4, 1),\n        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1),\n        \'reg_alpha\': trial.suggest_float(\'reg_alpha\', 1e-3, 10.0, log=True),\n        \'reg_lambda\': trial.suggest_float(\'reg_lambda\', 1e-3, 10.0, log=True),\n        "random_state" : SEED,\n        "verbosity" : -1,\n        "device" : "gpu",\n        "class_weight" : {\n            0 : 1,\n            1 : trial.suggest_float("class_weight", 1, 1200)\n        },\n    }\n\n    scores = []\n\n    for fold in data:\n\n        X_train,y_train,l_trai

In [16]:
study = optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler(seed=SEED))  

[I 2024-07-31 16:37:32,815] A new study created in memory with name: no-name-70c9e69e-7749-4f9e-a714-429e8a72f94a


In [17]:
study.optimize(objective,n_trials=500,show_progress_bar=True)

  0%|          | 0/500 [00:00<?, ?it/s]

[I 2024-07-31 16:37:34,709] Trial 0 finished with value: 0.10810775714639502 and parameters: {'n_estimators': 1935, 'eta': 0.28570714885887566, 'max_depth': 95, 'subsample': 0.7190609389379257, 'colsample_bytree': 0.40921304830970556, 'scale_pos_weight': 188.03742988310697, 'min_child_weight': 30, 'gamma': 8.661761457749352, 'lambda': 6.011150117432088}. Best is trial 0 with value: 0.10810775714639502.
[I 2024-07-31 16:37:35,585] Trial 1 finished with value: 0.14738291023007183 and parameters: {'n_estimators': 3570, 'eta': 0.01596950334578271, 'max_depth': 125, 'subsample': 0.8827098485602951, 'colsample_bytree': 0.44863737747479326, 'scale_pos_weight': 219.00813568131363, 'min_child_weight': 92, 'gamma': 3.0424224295953772, 'lambda': 5.247564316322379}. Best is trial 1 with value: 0.14738291023007183.
[I 2024-07-31 16:37:36,220] Trial 2 finished with value: 0.1357449269126692 and parameters: {'n_estimators': 2216, 'eta': 0.09445645065743215, 'max_depth': 80, 'subsample': 0.39764570245

In [18]:
models = []
scores = []

for fold in data:

    dtrain = fold['train']
    dval = fold['val']
    model = XGBEnsembler([top_k_xgb_models[score][0] for score in top_k_xgb_models])
    model.fit(dtrain)

    y_hat = model.predict(dval)
    scores.append(score(dval.get_label(), y_hat))

    models.append(model)

In [19]:
np.mean(scores)

0.16766216207430693

In [20]:
np.std(scores)

0.00802418314891179

In [21]:
scores

[0.17552969760065104,
 0.17410248090858066,
 0.15346058329485462,
 0.17051344049119563,
 0.16470460807625267]

In [23]:
import joblib
import numpy as np

In [22]:
splits_1 = joblib.load("/home/abdelnour/Documents/projects/skin-cancer-detection/src/training/splits.pkl")

In [24]:
splits_2 = joblib.load("/home/abdelnour/Documents/projects/skin-cancer-detection/data/splits.pkl")

In [25]:
splits_1

[(array([     0,      2,      3, ..., 401055, 401056, 401057]),
  array([     1,      4,      6, ..., 401050, 401052, 401058])),
 (array([     0,      1,      2, ..., 401055, 401057, 401058]),
  array([     3,      7,      8, ..., 401033, 401038, 401056])),
 (array([     0,      1,      2, ..., 401056, 401057, 401058]),
  array([    14,     17,     21, ..., 401045, 401054, 401055])),
 (array([     1,      2,      3, ..., 401056, 401057, 401058]),
  array([     0,     10,     13, ..., 401046, 401047, 401051])),
 (array([     0,      1,      3, ..., 401055, 401056, 401058]),
  array([     2,      5,      9, ..., 401037, 401053, 401057]))]

In [30]:
for (train_idx_1,val_idx_1),(train_idx_2,val_idx_2) in zip(splits_1,splits_2):
    print(np.all(train_idx_1 == train_idx_2),np.all(val_idx_1 == val_idx_2))

True True
True True
True True
True True
True True
